In [1]:
import pdfplumber
import json
from copy import deepcopy

path = "~/Desktop/"
file = "StoreReports-1.pdf"
file_path = path+file

# all files are formated the same
PAGES = 5

In [2]:
class items:
    identified = False
    begin = 0.00
    delivery = 0.00
    end = 0.00
    used = 0.00
    cost = 0.00
    net = 0.00

    def __init__(self, name) -> str:
        self.name = name

    def checked(self, id) -> bool:
        self.identified = id

    def addData(self, begin, delivery, end, used, cost, net):
        self.begin = begin
        self.delivery = delivery
        self.end = end
        self.used = used
        self.cost = cost
        self.net = net


In [3]:
# list of all items 
drinks = {
    '591ml Pepsi' : False,
    '591ml Diet Pepsi' : False,
    '591ml Mountain Dew' : False,
    '591ml Diet Mtn Dew' : False,
    '591ml Aquafina' : False,
    '591ml Dr Pepper' : False,
    '591ml Mug Root Beer' : False,
    '591ml Schw Gngr Ale' : False,
    '591ml WCherry Pepsi' : False,
    '591ml Crush Orange' : False,
    '591ml Seven Up' : False,
    '591ml Brsk IceT' : False,
    '2Ltr Pepsi' : False,
    '2Ltr Diet Pepsi' : False,
    '2Ltr Mountain Dew' : False,
    '2Ltr Dt Mtn Dew' : False,
    '2Ltr Dr Pepper' : False,
    '2Ltr Mug Root Beer' : False,
    '2Ltr Schw Gngr Ale' : False,
    '2Ltr Crush Orange' : False,
    '2Ltr Brsk SwTea Lm' : False,
    '2Ltr Seven Up' : False,
    '355ml Pepsi' : False,
    '355ml Diet Pepsi' : False,
    '355ml Diet Mtn Dew' : False,
    '355ml Dr Pepper' : False,
    '355ml Schw Gngr Ale' : False,
    '355ml Brsk SwTea Lm' : False,
    '355ml Seven Up' : False,
    '12oz Welch Gr Soda' : False
}

cheese = {
    'Cheddar Cheese' : False,
    'Three Cheese Blend' : False,
    'String Cheese' : False,
    'Agropur Cheese' : False,
    '2 Cheese' : False,
    '3 Cheese' : False,
    'Feta Cheese' : False,
    'Philly Whipped Cream Chee' : False

}

dough = {
    'Dough Tray 10': False,
    'Dough Tray 12': False,
    'Dough Tray 14': False,
    'Dough Tray 16': False,
    'Chocolate Chip Cookie': False,
    'Choc Chip Brownie': False,
    'Cinnamon Pull Apart': False,
    'Thin Crust 14': False,
    'Gluten Free Crust': False
}

# There are two pepperonis in the log, needs work
meats = {
    'Pepperoni' : False,
    'Bacon' : False,
    'Salami' : False,
    'Italian Sausage' : False,
    'Anchovies' : False,
    'Grilled Chicken' : False,
    'Chicken Popper' : False,
    'Salami' : False,
    'Canadian Bacon' : False,
    'Philly Cheesesteak' : False,
    'Sliced Spicy Meatballs' : False,
    'Meatball' : False,
    'Beef' : False,
    'Sausage' : False,
    'Italian Sausage' : False,
    'Roasted Wings' : False,
    'Donair Topping' : False
}

produce = {
    'Banana Peppers' : False,
    'Black Olives' : False,
    'Pineapple Lg Can' : False,
    'Mushrooms' : False,
    'Fresh Spinach' : False,
    'Green Peppers' : False,
    'Onions' : False,
    'Tomatoes' : False,
    'Banana Peppers' : False,
    'Pepperoncini Peppers' : False,
    'Jalapeno Peppers' : False,
    'Green Olives' : False
}

sauces = {
    'Bulk Ranch Sauce': False,
    'Pizza Sauce': False,
    'Jug Garlic Sauce': False,
    'Garlic Sauce Cups': False,
    'Pizza Sauce Cups': False,
    'Cheese Sauce Cups': False,
    'Ranch Sauce Cups': False,
    'Barbecue Sauce Cups': False,
    'Buffalo Sauce Cups': False,
    'Honey Must Sauce Cups': False,
    'Blue Cheese Sauce Cups': False,
    'Barbecue Sauce': False,
    'Garlic Parmesan Sauce Jug': False,
    'Buffalo Sauce': False,
    'Pouch Honey Chptl': False,
    'BBQ Bulk': False,
    'Alfredo Sauce': False,
    'Spinach Alfredo Sauce': False,
    'Cream Cheese Flavored Ici': False,
    'Creamy Garlic Dipping Cup': False,
    'Donair Sauce': False,
    'Hellman\'s Bulk Ranch': False
}

seasoning = {
    'Italian Seasoning' : False,
    'Seasoning Packets' : False,
    'Parmesan' : False,
    'Crushed Red Pepper' : False,
    'Dustinator' : False
}

boxes = {
    'Pizza Box 10': False,
    'Pizza Box 12': False,
    'Pizza Box 14': False,
    'Pizza Box 16': False,
    'Pizza Slice Box': False
}

In [4]:
category_dict = {
    'BEVERAGES' : drinks,
    'CHEESE' : cheese,
    'MEATS' : meats,
    'PRODUCE' : produce,
    'DOUGH' : dough,
    'SAUCES' : sauces,
    'SEASONS/FLOUR' : seasoning,
    'BOXES' : boxes
}

inventory = {}

data_columns = ['Begin', 'Delivery', 'End', 'Used', 'Cost', '% of sales']


In [5]:
# creates a josn with all the data : category { items { data { 'Begin' : 0.00, ... }}}
# TODO Fix : only uses data from the last row
def logData(data_in):
    for cat in category_dict:
        items = {}
        for item in category_dict[cat]:
            data = {}
            for i in range(len(data_columns)):
                try :
                    x = deepcopy(float(data_in[i]))
                except:
                    # print(f'* ERROR * failed to convert {data_in[i]} data for {item} to float')
                    x = 0.00
                    continue
                
                data[data_columns[i]] = x
                
            items[item] = deepcopy(data)
            
        inventory[cat] = items
        

In [6]:
def extrator(path) -> str:
    category = ''
    detail_flag = False # lets program know when to start recording values
    detail_counter = 0
    details = [0.00, 0.00, 0.00, 0.00, 0.00, 0.00]

    for page in range(PAGES):
        with pdfplumber.open(path) as pdf:
            page = pdf.pages[page]
            text = page.extract_text()

        print(f'Reading : {page}')
        word = ''
        space_counter = 0

        for character in text:
            if ord(character) != 160:                                               # 160 is the hex value for ' '
                word += character
                space_counter = 0

            elif len(word) > 0: 
                word += ' '
                space_counter += 1
                if space_counter == 2:                                              # word ends if there are more than one space between words
                    word = word.strip()
                    if word in category_dict:                                       # a new category will only appear once all items have been read
                        for item in category:
                            if not category[item]:
                                print(f'** WARNING ** did not log {item}')

                        category = category_dict[word]

                    elif word in category:
                        detail_flag = True
                        if detail_counter == 6:                                     # Log all the items when inventory information is collected
                            logData(details)

                        detail_counter = 0
                        category[word] = True

                    elif detail_counter > 5:
                        detail_flag = False

                    elif detail_flag and detail_counter < 6:
                        details[detail_counter] = word
                        detail_counter += 1

                    word = ''    
                    space_counter = 0       

    print('File Complete')

In [9]:
print(inventory)

{'BEVERAGES': {'591ml Pepsi': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Diet Pepsi': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Mountain Dew': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Diet Mtn Dew': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Aquafina': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Dr Pepper': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Mug Root Beer': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml Schw Gngr Ale': {'Begin': 171.0, 'Delivery': 200.0, 'End': 0.0, 'Used': 371.0, 'Cost': 96.72, '% of sales': 0.61}, '591ml WCherry Pepsi': {'Begin'

In [8]:
extrator(file)
# TODO everything is saved with the values for Pizza box 16
# save data to file
with open('data/data.json', 'w', encoding='utf-8') as f:
    json.dump(inventory, f, ensure_ascii=False, indent=4)

Reading : <Page:1>
Reading : <Page:2>
Reading : <Page:3>
Reading : <Page:4>
Reading : <Page:5>
File Complete
